In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import json
import numpy 
import os
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline  
import pyarrow
import sys

from datetime import date
from dotenv import load_dotenv
from sqlalchemy import create_engine
from os import path
from typing import List,Dict, Tuple
from collections import defaultdict
pd.set_option("display.max_columns", None)

load_dotenv(verbose=True)
BIGQUERY_CREDENTIALS_PATH = os.environ.get('BIGQUERY_CREDENTIALS_PATH')
engine = create_engine('bigquery://bespoke-financial/ProdMetrcData', credentials_path=os.path.expanduser(BIGQUERY_CREDENTIALS_PATH))

sys.path.append(path.realpath(path.join(os.getcwd(), "../core")))
sys.path.append(path.realpath(path.join(os.getcwd(), "../../src")))
sys.path.append(path.realpath(path.join(os.getcwd(), "../../scripts")))

import create_queries
import prepare_data

from bespoke.inventory.analysis.shared import download_util, inventory_types
from bespoke.inventory.analysis import active_inventory_util as util
from bespoke.inventory.analysis import inventory_valuations_util as valuations_util

%load_ext autoreload
%autoreload 2

In [3]:
from underwriting import client_surveillance_jupyter

In [4]:
COMPANY_IDENTIFIER = ['GF']
TRANSFER_PACKAGES_START_DATE = '2020-01-01'
SALES_TRANSACTIONS_START_DATE = '2020-01-01'

In [5]:
#fetch download report and lisences
company_licenses_query = create_queries.create_company_licenses_query(COMPANY_IDENTIFIER)
company_download_summaries_query = create_queries.create_company_download_summaries_query(COMPANY_IDENTIFIER, TRANSFER_PACKAGES_START_DATE)

company_licenses_dataframe = pd.read_sql_query(company_licenses_query, engine)
company_download_summaries_dataframe = pd.read_sql_query(company_download_summaries_query, engine)

## Check download status summaries

In [6]:
license_numbers = company_download_summaries_dataframe['license_number'].unique()
download_summary_records = company_download_summaries_dataframe.to_dict('records')

In [7]:
license_numbers

array(['C12-0000191-LIC', 'C10-0000700-LIC', 'C10-0000805-LIC',
       'C10-0000824-LIC', 'C11-0000401-LIC', 'C10-0000117-LIC',
       'C10-0000224-LIC'], dtype=object)

In [8]:
#bad_download_history = client_surveillance_jupyter.check_company_license_download(license_numbers,download_summary_records)

In [9]:
#bad_download_history

## Choose license numbers

In [10]:
retail_license_numbers = ['C10-0000700-LIC','C10-0000805-LIC','C10-0000824-LIC','C10-0000117-LIC','C10-0000224-LIC']


In [11]:
distro_license_numbers = ['C11-0000401-LIC']

In [12]:
all_license_numbers = ['C12-0000191-LIC', 'C11-0000401-LIC', 'C10-0000805-LIC',
       'C10-0000824-LIC', 'C10-0000700-LIC', 'C10-0000117-LIC',
       'C10-0000224-LIC']

## Pull data

## distro inventory

In [13]:
company_inventory_packages_query = create_queries.create_company_inventory_packages_query(
    COMPANY_IDENTIFIER,
    include_quantity_zero=True,
    license_numbers=distro_license_numbers,
)
company_inventory_packages_dataframe = pd.read_sql_query(company_inventory_packages_query, engine)
df_inventory = company_inventory_packages_dataframe
df_inventory['license_number'].unique()

array(['C11-0000401-LIC'], dtype=object)

## retail incoming

In [14]:
company_incoming_transfer_packages_query = create_queries.create_company_incoming_transfer_packages_query(
    COMPANY_IDENTIFIER,
    TRANSFER_PACKAGES_START_DATE,
    license_numbers=retail_license_numbers,
)

company_incoming_transfer_packages_dataframe = pd.read_sql_query(company_incoming_transfer_packages_query, engine)
df_in_retail = company_incoming_transfer_packages_dataframe
df_in_retail['license_number'].unique()

array(['C10-0000824-LIC', 'C10-0000700-LIC', 'C10-0000805-LIC',
       'C10-0000224-LIC'], dtype=object)

## all incoming

In [15]:
all_incoming_query = create_queries.create_company_incoming_transfer_packages_query(
    COMPANY_IDENTIFIER,
    TRANSFER_PACKAGES_START_DATE,
    license_numbers=all_license_numbers,
)

df_in_all = pd.read_sql_query(all_incoming_query, engine)
df_in_all['license_number'].unique()

array(['C11-0000401-LIC', 'C10-0000824-LIC', 'C10-0000805-LIC',
       'C10-0000700-LIC', 'C10-0000224-LIC', 'C12-0000191-LIC'],
      dtype=object)

## distro incoming

In [16]:
distro_incoming_query = create_queries.create_company_incoming_transfer_packages_query(
    COMPANY_IDENTIFIER,
    TRANSFER_PACKAGES_START_DATE,
    license_numbers=distro_license_numbers,
)

df_in_distro = pd.read_sql_query(distro_incoming_query, engine)
df_in_distro['license_number'].unique()

array(['C11-0000401-LIC'], dtype=object)

## distro outgoing

In [17]:
distro_outgoing_query = create_queries.create_company_outgoing_transfer_packages_query(
    COMPANY_IDENTIFIER,
    TRANSFER_PACKAGES_START_DATE,
    license_numbers=distro_license_numbers,
)

df_out_distro = pd.read_sql_query(distro_outgoing_query, engine)
df_out_distro['license_number'].unique()

array(['C11-0000401-LIC'], dtype=object)

# look at distro incoming data

In [ ]:
df_in_distro['created_date'] = pd.to_datetime(df_in_distro['created_date'])
df_in_distro['create_year'] = [df_in_distro['created_date'][i].year for i in range(len(df_in_distro))]


In [ ]:
# how many from internal vs external?
df_in_distro[['create_year','delivery_type','package_id']].groupby(['create_year','delivery_type']).count()

In [ ]:
# who are they buying from?
df_in_distro[df_in_distro['delivery_type'] == 'INCOMING_FROM_VENDOR']['shipper_facility_license_number'].unique()

In [ ]:
# top parties they buy from in 2022
df_in_distro[df_in_distro['create_year'] == 2022][['shipper_facility_name','package_id']].\
groupby('shipper_facility_name').count().sort_values(by = 'package_id',ascending = False)

In [ ]:
# df_in_distro_2022 = df_in_distro[(df_in_distro['created_date'] >= '2022-06-01')&\
#                                  (~df_in_distro['received_datetime'].isna()&
#                                   (df_in_distro['receiver_wholesale_price'].isna()))]

In [ ]:
#df_in_distro_2022['product_name'].to_dict().values()

# look at distro outgoing data

In [ ]:
df_out_distro['created_date'] = pd.to_datetime(df_out_distro['created_date'])
df_out_distro['create_year'] = [df_out_distro['created_date'][i].year for i in range(len(df_out_distro))]


In [ ]:
df_out_distro_in = df_out_distro[df_out_distro['recipient_facility_license_number'].isin(retail_license_numbers+distro_license_numbers)]

In [ ]:
# outgoing to internal 
df_out_distro_in[['create_year','package_id']].groupby(['create_year']).count()

In [ ]:
df_out_distro_ex = df_out_distro[~df_out_distro['recipient_facility_license_number'].isin(retail_license_numbers+distro_license_numbers)]

In [ ]:
# outgoing to external
df_out_distro_ex[['create_year','package_id']].groupby(['create_year']).count()

In [ ]:
# outgoing to external in 2022 who are they?
df_out_distro_ex[df_out_distro_ex['create_year'] == 2022]['recipient_facility_license_number'].unique()

In [ ]:
# outgoing to external in 2022 who are they?
df_out_distro_ex[df_out_distro_ex['create_year'] == 2022][['recipient_facility_name','recipient_facility_license_number','package_id']].\
groupby(['recipient_facility_name','recipient_facility_license_number']).count().sort_values(by = 'package_id',ascending = False)

## external outgoing to retailers

In [ ]:
df_out_distro_ex_ret = df_out_distro_ex[df_out_distro_ex['recipient_facility_license_number'].isin(\
                                                                                                   ['C10-0000112-LIC','C10-0000612-LIC',\
                                                                                                    'C10-0000329-LIC','C10-0000241-LIC'])]

In [ ]:
df_out_distro_ex_ret['per_unit'] = df_out_distro_ex_ret['receiver_wholesale_price'] / df_out_distro_ex_ret['received_quantity']
df_out_distro_ex_ret = df_out_distro_ex_ret[df_out_distro_ex_ret['per_unit'] > 0.1]


In [ ]:
price_ext_ret = df_out_distro_ex_ret[['product_name','per_unit']].groupby('product_name').mean().reset_index()

In [ ]:
q_ext_ret = df_out_distro_ex_ret[['product_name','received_quantity']].groupby('product_name').sum().reset_index()

## internal outoing to retailers

In [ ]:
df_out_distro_in_ret = df_out_distro_in[(df_out_distro_in['create_year'] == 2022)&\
                                        (df_out_distro_in['recipient_facility_license_number'].isin(retail_license_numbers))]


In [ ]:
df_out_distro_in_ret['per_unit'] = df_out_distro_in_ret['receiver_wholesale_price'] / df_out_distro_in_ret['received_quantity']
df_out_distro_in_ret = df_out_distro_in_ret[df_out_distro_in_ret['per_unit'] > 0.1]


In [ ]:
price_in_ret = df_out_distro_in_ret[['product_name','per_unit']].groupby('product_name').mean().reset_index()

In [ ]:
q_in_ret = df_out_distro_in_ret[['product_name','received_quantity']].groupby('product_name').sum().reset_index()

## merge the two pricing

In [ ]:
test = price_in_ret.merge(price_ext_ret,on = ['product_name'],how = 'left').\
merge(q_in_ret,on = ['product_name'],how = 'left').\
merge(q_ext_ret,on = ['product_name'],how = 'left')




In [ ]:
test

In [ ]:
match = test[~(test['per_unit_x'].isna())&~(test['per_unit_y'].isna())]

In [ ]:
match.columns = ['product_name','per_unit_int','per_unit_ext','quantity_int','quantity_ext']

In [ ]:
match['price_diff'] = match['per_unit_int'] - match['per_unit_ext']

In [ ]:
match

In [ ]:

df_out_distro_in_ret[df_out_distro_in_ret['product_name'] == 'Paletas - Cherry Pie - Infused Blunt - 2.1g']

## retail sales

In [34]:
company_sales_receipts_with_transactions_query = create_queries.create_company_sales_receipts_with_transactions_query(
    COMPANY_IDENTIFIER,
    SALES_TRANSACTIONS_START_DATE,
    license_numbers=retail_license_numbers,
)
company_sales_receipts_with_transactions_dataframe = pd.read_sql_query(company_sales_receipts_with_transactions_query, engine)
company_sales_receipts_with_transactions_dataframe['license_number'].unique()



array(['C10-0000805-LIC', 'C10-0000824-LIC', 'C10-0000700-LIC',
       'C10-0000117-LIC', 'C10-0000224-LIC'], dtype=object)

## dedupe sales transactions

In [35]:
deduped_sales_receipts_with_transactions_dataframe = prepare_data.dedupe_sales_transactions(company_sales_receipts_with_transactions_dataframe)

In [36]:
deduped_sales_receipts_with_transactions_dataframe['sales_month'] = deduped_sales_receipts_with_transactions_dataframe['sales_datetime'].dt.strftime('%Y-%m')

In [37]:
#check we have all locations
deduped_sales_receipts_with_transactions_dataframe['license_number'].unique()

array(['C10-0000805-LIC', 'C10-0000824-LIC', 'C10-0000700-LIC',
       'C10-0000117-LIC', 'C10-0000224-LIC'], dtype=object)

In [38]:
df_sales_retail = deduped_sales_receipts_with_transactions_dataframe

# Inventory

In [18]:
today = date.today()
today

datetime.date(2022, 9, 21)

In [19]:
COMPANY_IDENTIFIER

['GF']

In [20]:
distro_license_numbers

['C11-0000401-LIC']

## 1. distro inventory + sales based (distro to retail)

In [29]:
inv_val_distro_to_retail,inv_all_distro_to_retail = client_surveillance_jupyter.calculate_inventory_valuation(df_in_retail,df_inventory,license_numbers,today)

#make sense for CPG


In [30]:
inv_val_distro_to_retail

,date,value,value_after_tax,total_incoming,total,coverage,license,legal_name
0,2022-09-21,1000124.01,1262656.57,819,1120,0.73,"[C12-0000191-LIC, C10-0000700-LIC, C10-0000805...","FLOR EAST BAY, LLC"


In [25]:
#df = df_out_distro[df_out_distro['recipient_facility_license_number'].isin(retail_license_numbers)]

In [31]:
inv_val_distro_to_retail['value'].values / inv_val_distro_to_retail['coverage'].values

array([1370032.890410959], dtype=object)

In [32]:
missing = inv_all_distro_to_retail[inv_all_distro_to_retail['per_unit_product'].isna()]

In [55]:
missing[['product_name','unit_of_measure','quantity']].groupby(['product_name','unit_of_measure']).sum().sort_values(by = 'quantity',ascending = False)




,,quantity
product_name,unit_of_measure,
Firesale Blackberry Diesel Sativa Preroll 0.7g,Each,13812.0
Firesale MAC Hybrid Preroll 0.7g,Each,13614.0
GRIZZLY PEAK - Indica Bone - 0.5g - 7PK,Each,3048.0
Firesale Mochi Runtz Hybrid 5-Shot Preroll Pack 3.5g,Each,2109.0
Firesale Panama Punch Sativa 5-Shot Prerolls Pack 3.5g,Each,2099.0
...,...,...
High Supply - Silver Haze - Sativa - Flower 3.5g,Each,1.0
High Supply - Sour Apple - Indica - Flower 14g,Each,1.0
High Supply - Sour Apple - Indica - Flower 3.5g,Each,1.0


In [47]:
missing[missing['product_name'] == 'GRIZZLY PEAK - Indica Bone - 0.5g - 7PK']

,license_number,package_id,package_label,type,packaged_date,last_modified_at,package_type,product_name,product_category_name,quantity,unit_of_measure,item_id,item_product_category_type,production_batch_number,source_production_batch_numbers,source_harvest_names,is_testing_sample,is_trade_sample,is_on_hold,archived_date,finished_date,per_unit_incoming,per_unit_product
1068,C11-0000401-LIC,18955488,1A4060300002A31000000953,active,2021-10-21,2021-10-25 18:39:32+00:00,Product,GRIZZLY PEAK - Indica Bone - 0.5g - 7PK,Pre-Roll Infused,3048.0,Each,3573113,Concentrate,IB7PK005,,7/27/21 - 200 Flower 1 - Club Soda - Wet Flowe...,False,False,False,None,None,NaN,NaN


In [ ]:
# search for these products from GF's retail sales
# when they come in? new product?
#retailers should be selling them, do they appear in transactions?


In [62]:
#df_sales_retail[df_sales_retail['tx_product_name'] == 'Firesale Panama Punch Sativa 5-Shot Prerolls Pack 3.5g	']

In [56]:
df_in_all[df_in_all['product_name'] == 'Firesale Blackberry Diesel Sativa Preroll 0.7g' ]

,delivery_type,license_number,manifest_number,created_date,received_datetime,shipment_transaction_type,shipper_facility_license_number,shipper_facility_name,recipient_facility_license_number,recipient_facility_name,shipment_type_name,shipment_transaction_type_1,package_id,package_label,type,source_package_labels,source_harvest_names,shipment_package_state,is_testing_sample,is_trade_sample,product_category_name,product_name,package_lab_results_status,shipper_wholesale_price,shipped_quantity,shipped_unit_of_measure,receiver_wholesale_price,received_quantity,received_unit_of_measure,item_unit_weight,item_unit_weight_unit_of_measure_name
301,INCOMING_FROM_VENDOR,C11-0000401-LIC,0004067038,2022-09-20,2022-09-20 21:24:19+00:00,Standard,CDPH-10002658,710 COMBINATOR CO.,C11-0000401-LIC,"FLOR X, INC.",Transfer,Standard,28977657,1A4060300002A31000001671,transfer,"1A40603000087FB000000575, 1A4060300009B2C00000...","Jealousy-Bloom 3-8/18/22, Planet Dosi R2 H3.3....",Accepted,False,False,Pre-Roll Leaf,Firesale Blackberry Diesel Sativa Preroll 0.7g,failed,NaN,13844.0,Each,NaN,13844.0,Each,0.7,Grams


In [57]:
df_in_all[df_in_all['product_name'] == 'Firesale MAC Hybrid Preroll 0.7g' ]

,delivery_type,license_number,manifest_number,created_date,received_datetime,shipment_transaction_type,shipper_facility_license_number,shipper_facility_name,recipient_facility_license_number,recipient_facility_name,shipment_type_name,shipment_transaction_type_1,package_id,package_label,type,source_package_labels,source_harvest_names,shipment_package_state,is_testing_sample,is_trade_sample,product_category_name,product_name,package_lab_results_status,shipper_wholesale_price,shipped_quantity,shipped_unit_of_measure,receiver_wholesale_price,received_quantity,received_unit_of_measure,item_unit_weight,item_unit_weight_unit_of_measure_name
943,INCOMING_FROM_VENDOR,C11-0000401-LIC,0004037159,2022-09-13,2022-09-14 19:32:05+00:00,Standard,CDPH-10002658,710 COMBINATOR CO.,C11-0000401-LIC,"FLOR X, INC.",Transfer,Standard,28808316,1A4060300002A31000001660,transfer,"1A4060300001451000001061, 1A406030000514500000...","Gary Payton - 20220412, High Society 89 - 2022...",Accepted,False,False,Pre-Roll Leaf,Firesale MAC Hybrid Preroll 0.7g,failed,NaN,13646.0,Each,NaN,13646.0,Each,0.7,Grams


In [58]:
df_in_all[df_in_all['product_name'] == 'GRIZZLY PEAK - Indica Bone - 0.5g - 7PK' ]

,delivery_type,license_number,manifest_number,created_date,received_datetime,shipment_transaction_type,shipper_facility_license_number,shipper_facility_name,recipient_facility_license_number,recipient_facility_name,shipment_type_name,shipment_transaction_type_1,package_id,package_label,type,source_package_labels,source_harvest_names,shipment_package_state,is_testing_sample,is_trade_sample,product_category_name,product_name,package_lab_results_status,shipper_wholesale_price,shipped_quantity,shipped_unit_of_measure,receiver_wholesale_price,received_quantity,received_unit_of_measure,item_unit_weight,item_unit_weight_unit_of_measure_name
19949,INCOMING_FROM_VENDOR,C11-0000401-LIC,0003204282,2022-03-14,2022-03-14 17:22:31+00:00,Standard,CDPH-10002658,710 COMBINATOR CO.,C11-0000401-LIC,"FLOR X, INC.",Transfer,Standard,23120182,1A4060300002A31000001180,transfer,"1A4060300003BCE000003637, 1A4060300014C0A00000...",10/11/21 - 35 Flower 3 - Ice Cream Runtz - Wet...,Accepted,False,False,Pre-Roll Infused,GRIZZLY PEAK - Indica Bone - 0.5g - 7PK,failed,NaN,2643.0,Each,NaN,2643.0,Each,3.5,Grams
26143,INCOMING_FROM_VENDOR,C11-0000401-LIC,0002973567,2022-01-19,2022-01-19 22:26:32+00:00,Standard,CDPH-10002658,710 COMBINATOR CO.,C11-0000401-LIC,"FLOR X, INC.",Transfer,Standard,21494383,1A4060300002A31000001014,transfer,"1A4060300003091000000191, 1A4060300014C0A00000...",11/11/21 - 200 Flower 6 - Ice Cream Cake #5 - ...,Accepted,False,False,Pre-Roll Infused,GRIZZLY PEAK - Indica Bone - 0.5g - 7PK,failed,NaN,2215.0,Each,NaN,2215.0,Each,3.5,Grams
35398,INCOMING_FROM_VENDOR,C11-0000401-LIC,0002607118,2021-10-21,2021-10-21 23:11:25+00:00,Standard,CDPH-10002658,710 COMBINATOR CO.,C11-0000401-LIC,"FLOR X, INC.",Transfer,Standard,18955488,1A4060300002A31000000953,transfer,"1A4060300003091000000138, 1A4060300014C0A00000...",7/27/21 - 200 Flower 1 - Club Soda - Wet Flowe...,Accepted,False,False,Pre-Roll Infused,GRIZZLY PEAK - Indica Bone - 0.5g - 7PK,failed,NaN,3061.0,Each,NaN,3061.0,Each,3.5,Grams
36684,INCOMING_FROM_VENDOR,C11-0000401-LIC,0002559268,2021-10-11,2021-10-11 21:56:27+00:00,Standard,CDPH-10002658,710 COMBINATOR CO.,C11-0000401-LIC,"FLOR X, INC.",Transfer,Standard,18623661,1A4060300002A31000000951,transfer,"1A4060300003091000000138, 1A4060300014C0A00000...",7/27/21 - 200 Flower 1 - Club Soda - Wet Flowe...,Accepted,False,False,Pre-Roll Infused,GRIZZLY PEAK - Indica Bone - 0.5g - 7PK,failed,NaN,3202.0,Each,NaN,3202.0,Each,3.5,Grams


In [59]:
df_in_all[df_in_all['product_name'] == 'Firesale Mochi Runtz Hybrid 5-Shot Preroll Pack 3.5g' ]

,delivery_type,license_number,manifest_number,created_date,received_datetime,shipment_transaction_type,shipper_facility_license_number,shipper_facility_name,recipient_facility_license_number,recipient_facility_name,shipment_type_name,shipment_transaction_type_1,package_id,package_label,type,source_package_labels,source_harvest_names,shipment_package_state,is_testing_sample,is_trade_sample,product_category_name,product_name,package_lab_results_status,shipper_wholesale_price,shipped_quantity,shipped_unit_of_measure,receiver_wholesale_price,received_quantity,received_unit_of_measure,item_unit_weight,item_unit_weight_unit_of_measure_name
944,INCOMING_FROM_VENDOR,C11-0000401-LIC,0004037159,2022-09-13,2022-09-14 19:32:05+00:00,Standard,CDPH-10002658,710 COMBINATOR CO.,C11-0000401-LIC,"FLOR X, INC.",Transfer,Standard,28816615,1A4060300002A31000001669,transfer,"1A4060300006A50000019481, 1A4060300009B2C00000...","Jealousy-Bloom 3-8/18/22, Mochi Runtz-Bloom 1-...",Accepted,False,False,Pre-Roll Flower,Firesale Mochi Runtz Hybrid 5-Shot Preroll Pac...,failed,NaN,2122.0,Each,NaN,2122.0,Each,3.5,Grams


In [61]:
df_in_all[df_in_all['product_name'] == 'Firesale Panama Punch Sativa 5-Shot Prerolls Pack 3.5g' ]

,delivery_type,license_number,manifest_number,created_date,received_datetime,shipment_transaction_type,shipper_facility_license_number,shipper_facility_name,recipient_facility_license_number,recipient_facility_name,shipment_type_name,shipment_transaction_type_1,package_id,package_label,type,source_package_labels,source_harvest_names,shipment_package_state,is_testing_sample,is_trade_sample,product_category_name,product_name,package_lab_results_status,shipper_wholesale_price,shipped_quantity,shipped_unit_of_measure,receiver_wholesale_price,received_quantity,received_unit_of_measure,item_unit_weight,item_unit_weight_unit_of_measure_name
1039,INCOMING_FROM_VENDOR,C11-0000401-LIC,0004028819,2022-09-12,2022-09-13 20:22:51+00:00,Standard,CDPH-10002658,710 COMBINATOR CO.,C11-0000401-LIC,"FLOR X, INC.",Transfer,Standard,28748276,1A4060300002A31000001668,transfer,"1A4060300006A50000019481, 1A4060300009B2C00000...","Purple P-5, Purple Punch-5, Purple-3, Purple-5...",Accepted,False,False,Pre-Roll Flower,Firesale Panama Punch Sativa 5-Shot Prerolls P...,failed,NaN,2112.0,Each,NaN,2112.0,Each,3.5,Grams


## 2. distro inventory + cost based (cost to distro)

In [ ]:
inv_val_distro_cost,inv_all_distro_cost = client_surveillance_jupyter.calculate_inventory_valuation(df_in_distro,df_inventory,license_numbers,today)




In [ ]:
inv_val_distro_cost

In [ ]:
inv_val_distro_cost['value'].values / inv_val_distro_cost['coverage'].values

## 3. blended: distro inventory + all incoming data

In [ ]:
inv_val_all,inv_all = client_surveillance_jupyter.calculate_inventory_valuation(df_in_all,df_inventory,license_numbers,today)




In [ ]:
inv_val_all

In [ ]:
inv_val_all['value'].values / inv_val_all['coverage'].values